## **ANALYSIS_masters_make_expected_het**

### GOALS of this script:
* make the expected het map for masters thesis
* calculate expected het for all X landraces 
* plot the value and abbreviation on the map 
* final figure desgin

### **Import packages**

In [ ]:
import numpy as np
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import allel
import random
import h5py
import bcolz
import csv
from itertools import chain
import os
from collections import Counter
import plotly.graph_objects as go

### **Load the data**

In [ ]:
callset = allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf')

In [ ]:
#callset

In [ ]:
sorted(callset.keys())

In [ ]:
#get the samples in the .vcf
samples= callset["samples"]
samples

In [ ]:
#total number of the samples inn the vcf
len(samples)

In [ ]:
#put all the samples in a Dataframe
sample_subset = pd.DataFrame(data=samples, columns = ['sample_names'])
#sample_subset

### **Get the individuals for all 35 European landraces**

#### **Mayer_2017_landraces** - 952 individuals based on 35 landraces

In [ ]:
#all mayer_2017 sample names
mayer_2017_landraces_samples=sample_subset[sample_subset.sample_names.str.contains("_2017_landraces")]

In [ ]:
#number of all Mayer_2017_landraces
print(len(mayer_2017_landraces_samples))
#mayer_2017_landraces_samples

In [ ]:
mayer_2017_landraces_samples

In [ ]:
#add the populations to the individuals
for index, row in mayer_2017_landraces_samples.iterrows():
    #print(row['sample_names'])
    mayer_string=row['sample_names']#take the sample names row
    #print(mayer_string) 
    mayer_substring=mayer_string[0:2] #take the first two letters of the sample names
    #print(mayer_substring)
    mayer_2017_landraces_samples.at[index,'Population'] = mayer_substring #+ "_mayer_2017" #add the dataset info and make the Population column 

In [ ]:
mayer_2017_landraces_samples

In [ ]:
#get all the unique population that you have assigned and check the number 
populations_mayer = mayer_2017_landraces_samples.Population.unique()
print(len(populations_mayer))
print(populations_mayer)

In [ ]:
#Counter(mayer_2017_landraces_samples.Population)

#### **Unterseer_2016_landraces** - 906 individuals based on 38 european landraces

In [ ]:
#all Unterseer_2016_landraces sample names
unterseer_2016_landraces_samples=sample_subset[sample_subset.sample_names.str.contains("_2016_landraces")]
#unterseer_2016_landraces_samples

In [ ]:
#number of all Unterseer_2016_landraces samples
print(len(unterseer_2016_landraces_samples))
#unterseer_2016_landraces_samples

In [ ]:
#add the populations to all the individuals 
for index, row in unterseer_2016_landraces_samples.iterrows():
    #print(row['sample_names'])
    unterseer_string=row['sample_names'] #take the sample names row
    #print(unterseer_string) 
    unterseer_substring=unterseer_string[0:2] #take the first two letters of the sample names
    #print(unterseer_substring)
    unterseer_2016_landraces_samples.at[index,'Population'] = unterseer_substring #+ "_unterseer_2016" #add the dataset info and make the Population column

In [ ]:
unterseer_2016_landraces_samples

In [ ]:
#get all the unique population that you have assigned and check the number 
populations_unterseer = unterseer_2016_landraces_samples.Population.unique()
print(len(populations_unterseer))
print(populations_unterseer)

In [ ]:
#Counter(unterseer_2016_landraces_samples.Population)

**Select the three unique populations from the Unterseer_2016 landraces**

In [ ]:
unique_unterseer_2016_landraces_samples=unterseer_2016_landraces_samples.loc[unterseer_2016_landraces_samples['Population'].isin(['AL','SO','FL'])]

In [ ]:
unique_unterseer_2016_landraces_samples

In [ ]:
#unique_unterseer_2016_landraces_samples[unique_unterseer_2016_landraces_samples['Population']=="AL"]

In [ ]:
#unique_unterseer_2016_landraces_samples[unique_unterseer_2016_landraces_samples['Population']=="FL"]

In [ ]:
#unique_unterseer_2016_landraces_samples[unique_unterseer_2016_landraces_samples['Population']=="SO"]

In [ ]:
pd.unique(unique_unterseer_2016_landraces_samples['Population'])

#### **Combine the subset of samples**

In [ ]:
#combine the subset of samples
filtered_subset_samples_landraces_and_Dh_and_elite=pd.concat([unique_unterseer_2016_landraces_samples,mayer_2017_landraces_samples])

In [ ]:
filtered_subset_samples_landraces_and_Dh_and_elite

In [ ]:
#Counter(filtered_subset_samples_landraces_and_Dh_and_elite["Population"])

In [ ]:
#summarize the lists to check the lenghts 
952+72

In [ ]:
populations=pd.unique(filtered_subset_samples_landraces_and_Dh_and_elite["Population"])
print(populations)
print(len(populations))

### **Get the individuals for each of the populations**

In [ ]:
two_row_dataframe = {}
splits = list(filtered_subset_samples_landraces_and_Dh_and_elite.groupby("Population"))
populations_reordered=[]
for i in range(38):
    two_row_dataframe[i] = [] 
    # view splitted dataframe
    pop=splits[i][1]
    unique_pop=list(pd.unique(pop["Population"]))
    populations_reordered.append(unique_pop)
    #print(splits[i][1])
    two_row_dataframe[i]=splits[i][1]

In [ ]:
#two_row_dataframe[1]

In [ ]:
flat_list_populations = [item for sublist in populations_reordered for item in sublist]

In [ ]:
populations_landraces_38_reordered=np.array(flat_list_populations)
populations_landraces_38_reordered

In [ ]:
first_row_dataframe={} 
second_row_dataframe={}
for i in range(38):
    first_row_dataframe[i]=[]
    second_row_dataframe[i]=[]
    # view splitted dataframe
    #print(two_row_dataframe[i].sample_names)
    #print(two_row_dataframe[i].Population)
    first_row_dataframe[i]=two_row_dataframe[i].sample_names
    second_row_dataframe[i]=two_row_dataframe[i].Population

In [ ]:
#first_row_dataframe

In [ ]:
#second_row_dataframe

### **Calculate expected heterozygousity**

In [ ]:
#set up lists
expected_het_list=[]
#expected_het_pos_list=[]
#expected_het_chrom_list=[]

In [ ]:
for i in range(38):
    #print(i)
    
    #load the indviduals of the population and the name of the population
    individuals=first_row_dataframe[i]
    #print(individuals)
    individuals_pop=populations_landraces_38_reordered[i]
    #print(individuals_pop)
    
    #load the data for the population
    loc_samples_population= allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf', samples=individuals)
    
    #get the genotype data for the population
    geno_samples_population=allel.allel.GenotypeArray(loc_samples_population['calldata/GT'])
    
    #get the allel count array for the population
    geno_samples_population_allel_count=geno_samples_population.count_alleles()
    
    #boolean to check if the SNPs segregate
    is_seg = geno_samples_population_allel_count.is_segregating()[:]
    
    #apply the seg boolean to the genotype data
    loc_samples_population_is_seg=geno_samples_population #.compress(is_seg, axis=0) #geno_samples_population 
    
    #filter the positions 
    #loc_samples_population_pos=loc_samples_population['variants/POS']
    #loc_samples_population_pos_is_seg=loc_samples_population_pos.compress(is_seg, axis=0) #loc_samples_population_pos
    
    #filter the chromosomes 
    #loc_samples_population_chrom=loc_samples_population['variants/CHROM']
    #loc_samples_population_chrom_is_seg=loc_samples_population_chrom.compress(is_seg, axis=0) #loc_samples_population_chrom
    
    #make allel frequency array
    loc_samples_population_allel_freq = loc_samples_population_is_seg.count_alleles().to_frequencies()
    
    #calculate 
    het_expected_calc=allel.heterozygosity_expected(loc_samples_population_allel_freq, ploidy=2)

    #append elements to lists
    expected_het_list.append(het_expected_calc)
    #expected_het_pos_list.append(loc_samples_population_pos_is_seg)
    #expected_het_chrom_list.append(loc_samples_population_chrom_is_seg)
    
    #print the calculations 
    print("Expected heterozygoisty for %(a)s was calculated" % {'a': individuals_pop})
    

In [ ]:
expected_het_list

### **Calculate the mean expected heterozygousity**

In [ ]:
mean_expected_het_frame=pd.DataFrame()

In [ ]:
for k in range(38):
    individuals_pop=populations_landraces_38_reordered[k]
    calc_mean=np.nanmean(expected_het_list[k])
    
    mean_expected_het_frame = mean_expected_het_frame.append({'Population': individuals_pop, 'mean expected heterozygosity': calc_mean}, ignore_index=True)
    
    print("Mean observed heterozygoisty for %(a)s was calculated" % {'a':individuals_pop})

In [ ]:
mean_expected_het_frame=mean_expected_het_frame.set_index('Population')
mean_expected_het_frame

### **Prepare table for plotting the values on the map**

In [ ]:
#write the expected heterozygosity column to file 
#mean_expected_het_frame.to_csv("mean_expected_het_frame_all_pos.csv")

In [ ]:
#load the full metadata file
meta_full_landraces=pd.read_csv('meta_full_landraces_with_PCA_shapes_reordered_new_colorpalette.csv') 
meta_full_landraces

In [ ]:
#subselect the 38 unique European landraces 
landraces_unique_38=meta_full_landraces.loc[meta_full_landraces['Abb.'].isin(populations_landraces_38_reordered)].set_index('Abb.')
landraces_unique_38

In [ ]:
len(landraces_unique_38["Full name"])

In [ ]:
#add the expected het value into the table 
merged = landraces_unique_38.merge(mean_expected_het_frame, left_index=True, right_index=True, how='inner')

In [ ]:
merged

#### **Edit the expected het values**

In [ ]:
#rounded up value for three numbers after dot
merged_rounded=merged.round({"mean expected heterozygosity":3})

In [ ]:
#merged_rounded

In [ ]:
#make value into type string
merged_rounded=merged_rounded.astype({"mean expected heterozygosity": str})

In [ ]:
#merged_rounded

### **Prepare table for plotting the values on the map**

In [ ]:
#get the index, here to population abbreviations 
merged_rounded_index=merged_rounded.index

In [ ]:
merged_rounded_index

In [ ]:
fig = go.Figure(data=go.Scattergeo(
        lon = merged_rounded['Longitude'], #londitude
        lat = merged_rounded['Latitude'], #latitude
        mode="markers+text", #print both the markers & the text
        text= merged_rounded_index + " " + merged_rounded['mean expected heterozygosity'], #text is the pop abbreviations & rounded value
        marker_color = merged['hex_col'], #customize the color of the marker
        marker = dict(
            symbol = merged['shape'], #customize the shape of the marker
            size = 15, #meta_full_landraces['size'], #customize the size of the marker
            )
        ))

fig.update_geos( #customize the visualization of the map
    visible=False, resolution=50, scope="europe",
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="Blue",
    showland=True, landcolor="LightGrey",
)

fig.update_traces(textfont_size=15, textposition= "middle right" 
    )#change text size & location 

#customize the fig size & the look of the hover text box
fig.update_layout(
    
    height=1000,
    margin={"r":0,"t":0,"l":0,"b":0},
)
#write the figure to html
fig.write_html('map_europe_for_master.html', auto_open=True)

#write the figure to file 
#fig.write_image('map_europe_for_poster_for_inkspace_check.pdf',  format='.pdf',width=3000, height=3000)